# Image Inpainting with Runware API

In [ ]:
import base64
import requests
import json
from PIL import Image
from io import BytesIO
from google.colab import userdata

In [ ]:
def image_inpainting(positive_prompt,
                     negative_prompt,
                     seed_image,
                     mask_image,
                     width=1280, height=832, # output image dimensions
                     model="civitai:620368@860423", # RealVisXL-inpainting
                     number_results=1):

    url = "https://api.runware.ai/v1"

    # Create the payload
    payload = [
        {
            "taskType": "authentication",
            "apiKey": userdata.get('runware_key')
        },
        {
            "taskType": "imageInference",
            "taskUUID": "39d7207a-87ef-4c93-8082-1431f9c1dc97",
            "positivePrompt": positive_prompt,
            "negativePrompt": negative_prompt,
            "width": width,
            "height": height,
            "steps": 30,
            "CFGScale": 9,
            "seedImage": seed_image,
            "maskImage": mask_image,
            "model": model,
            "strength": 0.98,
            "numberResults": number_results,
            "includeCost": True
        }
    ]

    # Set headers
    headers = {
        "Content-Type": "application/json"
    }

    # Make the POST request
    response = requests.post(url, headers=headers, data=json.dumps(payload))

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Convert images to Base 64 string

In [ ]:
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        base64_string = base64.b64encode(image_file.read()).decode("utf-8")
    return base64_string

In [ ]:
# convert seed image to Base64 string
seed_image_path = "car1.jpg"
seed_image_base64_str = image_to_base64(seed_image_path)

# convert mask image to Base64 string
mask_image_path = "car1_mask.jpg"
mask_image_base64_str = image_to_base64(mask_image_path)

# Generate image with inpainting

In [ ]:
pos_prompt = """
(realistic photograph of a squirrel:1.2),
high quality, masterpiece, 8k
"""

neg_prompt = """
worst quality, low quality, normal quality, lowres, low details,
oversaturated, undersaturated, overexposed, underexposed, grayscale,
bad photo, bad photography
"""

result = image_inpainting(positive_prompt = pos_prompt,
                          negative_prompt = neg_prompt,
                          seed_image = seed_image_base64_str,
                          mask_image = mask_image_base64_str)

In [ ]:
result

In [ ]:
# read image from URL
response = requests.get(result['data'][0]['imageURL'])
output_image = Image.open(BytesIO(response.content))

In [ ]:
# display image
output_image